In [ ]:
%matplotlib inline

# Assuming we are in the notebooks directory, we need to move one up:
%cd ..

from __future__ import print_function

import os
import us
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt


# Arizona

In [ ]:
state_name = 'AZ'
best_path = glob.glob('./clustered/best_stats/' + state_name +'*')[0]
best_basename = os.path.basename(best_path)
filename, file_extension = os.path.splitext(best_basename)
state = gpd.read_file(glob.glob('./clustered/geojson_results/' + filename +'.geojson')[0])

In [ ]:
state.head()

In [ ]:
mini = state[['Predicted 2015 Population', 'geometry', 'Cluster', 'Congressional District', 'GEOID']]

In [ ]:
mini['area'] = mini['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)

In [ ]:
# GEOID Structure is defined as
# STATE + COUNTY + TRACT + BLOCK = 2 + 3 + 6 + 4 = 15
# see: https://www.census.gov/geo/reference/geoidentifiers.html
def get_tract(row):
#     print(type(row['GEOID']))
    return row['GEOID'][5:11]
mini['Tract'] = mini.apply(get_tract, axis=1)

In [ ]:
tracts = mini.dissolve(by='Tract')

In [ ]:
tracts.head()

In [ ]:
mini = mini[['Predicted 2015 Population', 'area', 'Tract']]

In [ ]:
mini.head()

In [ ]:
pop = mini.groupby('Tract').sum()
pop['Tract'] = pop.index
pop.head()

In [ ]:
del tracts['Predicted 2015 Population']

In [ ]:
del tracts['area']

In [ ]:
tracts['Tract'] = tracts.index

In [ ]:
tracts.head()

In [ ]:
tracts = tracts.merge(pop, on='Tract')

In [ ]:
tracts['Population per square km'] = tracts['Predicted 2015 Population'] / tracts['area'] 

In [ ]:
tracts.head()

In [ ]:
tracts.to_file('./fig/shapefile/results/pop_' + state_name + '_by_tract.shp', driver="ESRI Shapefile")

# Nebraska

In [180]:
state_name = 'NE'
best_path = glob.glob('./clustered/best_stats/' + state_name +'*')[0]
best_basename = os.path.basename(best_path)
filename, file_extension = os.path.splitext(best_basename)
state = gpd.read_file(glob.glob('./clustered/geojson_results/' + filename +'.geojson')[0])

In [182]:
mini = state[['Predicted 2015 Population', 'geometry', 'Cluster', 'Congressional District', 'GEOID']]

In [183]:
mini['area'] = mini['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [184]:
# GEOID Structure is defined as
# STATE + COUNTY + TRACT + BLOCK = 2 + 3 + 6 + 4 = 15
# see: https://www.census.gov/geo/reference/geoidentifiers.html
def get_tract(row):
#     print(type(row['GEOID']))
    return row['GEOID'][5:11]
mini['Tract'] = mini.apply(get_tract, axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [185]:
tracts = mini.dissolve(by='Tract')

In [186]:
mini = mini[['Predicted 2015 Population', 'area', 'Tract']]

In [187]:
pop = mini.groupby('Tract').sum()
pop['Tract'] = pop.index

In [188]:
del tracts['Predicted 2015 Population']

In [189]:
del tracts['area']

In [190]:
tracts['Tract'] = tracts.index
tracts = tracts.merge(pop, on='Tract')
tracts['Population per square km'] = tracts['Predicted 2015 Population'] / tracts['area'] 
tracts.to_file('./fig/shapefile/results/pop_' + state_name + '_by_tract.shp', driver="ESRI Shapefile")

# Major Cities in Arizona and Nebraska

In [ ]:
cities = gpd.read_file('./fig/shapefile/citiesx010g_shp_nt00962/citiesx010g.shp')
# From: https://catalog.data.gov/dataset/usgs-small-scale-dataset-cities-and-towns-of-the-united-states-201403-shapefile

In [ ]:
cities.head()

In [ ]:
az_ne_cities = cities[(cities['STATE'] == 'NE') | (cities['STATE'] == 'AZ')]

In [ ]:
az_ne_cities = az_ne_cities[(az_ne_cities['NAME'] == 'Tucson') | (az_ne_cities['NAME'] == 'Phoenix') |  (az_ne_cities['NAME'] == 'Omaha')]

In [ ]:
az_ne_cities.to_file('./fig/shapefile/cities/az_ne_cities.shp', driver="ESRI Shapefile")

# Confressional Districts for Arizona and Nebraska

In [ ]:
from default_values import usa_shapefile_path
usa = gpd.read_file(usa_shapefile_path)

In [ ]:
az_ne_cong_dist = usa[(usa['STATEFP'].apply(int) == int(us.states.lookup('NE').fips)) |
    (usa['STATEFP'].apply(int) == int(us.states.lookup('AZ').fips))]

In [ ]:
az_ne_cong_dist.plot()

In [ ]:
az_ne_cong_dist.to_file('./fig/shapefile/cong_dist/az_ne_cong_dist.shp', driver="ESRI Shapefile")